<a href="https://colab.research.google.com/github/carlos-alves-one/Comparing-Two-AI-Models/blob/master/Comparing_Two_AI_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 18/08/2022**

In [ ]:
!git init

Reinitialized existing Git repository in /content/.git/


In [ ]:
!git status

On branch master
nothing to commit, working tree clean


In [ ]:
!git add .

In [ ]:
!git commit -m "Initial commit and headers"

[master (root-commit) 275342b] Initial commit and headers
 19 files changed, 50771 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/gce
 create mode 100644 .config/logs/2022.08.15/13.43.32.260465.log
 create mode 100644 .config/logs/2022.08.15/13.43.53.840714.log
 create mode 100644 .config/logs/2022.08.15/13.44.12.999341.log
 create mode 100644 .config/logs/2022.08.15/13.44.20.441098.log
 create mode 100644 .config/logs/2022.08.15/13.44.40.370262.log
 create mode 100644 .config/logs/2022.08.15/13.44.41.125604.log
 create mode 100755 sample_data/README.md
 create mode 100755 sample_data/anscombe.json
 create mode 100644 sample_data/california_housing_test.csv
 create mode 100644 sample_data/calif

In [ ]:
!git config --global user.email "carlos.alves.one@gmail.com"

In [ ]:
!git config --global user.name "Carlos Alves"

In [ ]:
!git push -u origin main

error: src refspec main does not match any.
error: failed to push some refs to 'git@github.com:carlos-alves-one/Harvard-Medical-AI.git'
